In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
from collections import defaultdict
import json
import re
import requests
import sys
import random
import requests
import tabulate
from datetime import datetime
import schedule
import time

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')
cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
#Get work logs
sql = f'''
SELECT
  PUBLIC.QAEVENTS.user_email :: string EMAIL,
  PUBLIC.QAEVENTS.metadata: pvids PVIDS,
  (PUBLIC.QAEVENTS.metadata: totalMillisElapsed)/100 TIMESPENTSEC,
  --  PUBLIC.QAEVENTS.metadata: fieldBefore BEFORE,
  --  PUBLIC.QAEVENTS.metadata: fieldCurrent AFTER,
  PUBLIC.QAEVENTS.event_type,
  PUBLIC.QAEVENTS.event_at,
  cast (PUBLIC.QAEVENTS.event_at as DATE) AS EventDate,
  --  PUBLIC.QAEVENTS.metadata: fullStoryEvent :: string,
  --  PUBLIC.QAEVENTS.metadata: auditLevel :: string,
  --  PUBLIC.QAEVENTS.metadata: hintsAvailable,
  PUBLIC.QAEVENTS.metadata: domain SITE
FROM
  PUBLIC.QAEVENTS
WHERE
--        CAST(PUBLIC.QAEVENTS.event_at AS DATE) = CURRENT_DATE()
      CAST(PUBLIC.QAEVENTS.event_at AS DATE) BETWEEN '2022-11-21' AND '2022-11-28'
  AND PUBLIC.QAEVENTS.metadata :domain is not NULL
  AND PUBLIC.QAEVENTS.event_type = 'submit_description_speed_audit'
'''
cs.execute(sql)
worklogdf = cs.fetch_pandas_all()


In [4]:
#worklogdf.to_csv('foriris.csv')

In [5]:
#Create QA or Teleworker classification column, convert timespent to minutes
worklogdf['LEVEL'] = np.where(worklogdf['EMAIL'].str.contains("contractors"), 'QA', np.where(worklogdf['EMAIL'].str.contains("telework"), 'TW', 'ScaleHQ'))
worklogdf['TIMESPENTSEC'] = worklogdf['TIMESPENTSEC'].astype(int)
worklogdf['TIMESPENTSEC'] = (worklogdf['TIMESPENTSEC'] / 60)
#worklogdf = worklogdf.astype({"TIMESPENTSEC": int})
#worklogdf

In [6]:
#Summarize reports
qacount = worklogdf.groupby(['EVENTDATE','LEVEL'])['EMAIL'].nunique()
qacount

EVENTDATE   LEVEL  
2022-11-21  QA         2
            TW         2
2022-11-22  QA         6
            ScaleHQ    1
            TW         2
2022-11-23  QA         6
            TW         1
2022-11-24  QA         5
            TW         2
2022-11-25  QA         5
            TW         1
2022-11-28  QA         1
Name: EMAIL, dtype: int64

In [7]:
descount = worklogdf.groupby(['LEVEL'])['EVENTDATE'].count()
descount = descount.to_frame()
descount.columns = ['Descriptions']
descount
#desccount.sort_values('EVENTDATE','LEVEL', ascending=False)

,Descriptions
LEVEL,
QA,8812
ScaleHQ,16
TW,8545


In [8]:
#get descriptions count
tot = descount['Descriptions'].sum()
tot

17373

In [9]:
#get sites count
tmpct=[]
sitecount=0
for i in range(0, len(worklogdf['SITE'])):
    if worklogdf['SITE'][i] not in tmpct: 
        tmpct.append(worklogdf['SITE'][i])
        sitecount += 1
print(sitecount)

48


In [10]:
#get sites list
sitedf = worklogdf.groupby(['SITE','EVENTDATE'])['PVIDS'].count()
sitedf.to_frame()
sitedf.to_csv('pp_site_list.csv')

In [11]:
#get description count by QA
qadescount = worklogdf.groupby(['EMAIL','LEVEL'])['EVENTDATE'].count()
qadescount = qadescount.to_frame()
qadescount
#qadescount = qadescount.sort_values(by='Descriptions', ascending=False)
#qdlite = qadescount.drop(['EVENTDATE'], axis=1)

,,EVENTDATE
EMAIL,LEVEL,
ana.esparza@contractors.scale.com,QA,23
caterina.bourg@contractors.scale.com,QA,1722
dana.celeste@contractors.scale.com,QA,51
erwin.tudla@teleworkph-mails.com,TW,4580
german.toledo@contractors.scale.com,QA,2736
jesus.olvera@contractors.scale.com,QA,3
joyce.palma@contractors.scale.com,QA,587
karla.nunez@contractors.scale.com,QA,2759
kaushik.mahorker@scale.com,ScaleHQ,16


In [ ]:
#get time spent by QA
qatime = worklogdf.groupby(['EMAIL','EVENTDATE','LEVEL'])['TIMESPENTSEC'].sum()
qatime